In [4]:
# coding: utf-8

import pandas as pd
from datetime import timedelta
import datetime

In [19]:
#---------------------------------------------------------
print('start',datetime.datetime.now())

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('~/git/data/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    nrows=600000 #Skip dates before 2016-08-01
                    )

start 2017-11-26 21:13:33.243731


In [20]:
#---------------------------------------------------------
print('finish loading',datetime.datetime.now())

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

finish loading 2017-11-26 21:13:35.408308


In [21]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
#---------------------------------------------------------
print('finish data',datetime.datetime.now())
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

finish data 2017-11-26 21:13:35.762635


In [22]:
#---------------------------------------------------------
print('finish sep',datetime.datetime.now())
del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

finish sep 2017-11-26 21:13:36.187648


In [26]:
#---------------------------------------------------------
print('get lastdate',datetime.datetime.now())
#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)




get lastdate 2017-11-26 21:21:41.534475


In [ ]:
#-------------------------------------------
# normalization
se_max = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].quantile(0.9,interpolation='lower').to_frame('semax')
se_max.reset_index(inplace=True)

se_min = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].quantile(0.1,interpolation='lower').to_frame('semin')
se_min.reset_index(inplace=True)

new_train = pd.merge(train, se_max, how='left', on=['item_nbr','store_nbr','dow'])
new_train = pd.merge(new_train, se_min, how='left', on=['item_nbr','store_nbr','dow'])
new_train['unit_sales'] = (new_train['semax'] - new_train['unit_sales'])/(new_train['semax'] - new_train['semin'])

new_train.loc[(train.unit_sales<0),'unit_sales'] = 0
new_train.loc[(train.unit_sales>1),'unit_sales'] = 1

In [ ]:
#---------------------------------------------------------
print('get ma_dw ma_wk',datetime.datetime.now())
#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

In [ ]:
#---------------------------------------------------------
print('get MA',datetime.datetime.now())
del tmp,tmpg,train

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)

In [ ]:
#---------------------------------------------------------
print('get median of MA',datetime.datetime.now())
#Load test
test = pd.read_csv('~/git/data/test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])
test = pd.merge(test, se_max, how='left', on=['item_nbr','store_nbr','dow'])
test = pd.merge(test, se_min, how='left', on=['item_nbr','store_nbr','dow'])

In [ ]:
#---------------------------------------------------------
print('get test',datetime.datetime.now())
del ma_is, ma_wk, ma_dw, se_max, se_min

#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)

test['unit_sales'] = test['semax'] - test['unit_sales'] * (test['semax'] - test['semin'])
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

In [ ]:
#---------------------------------------------------------
print('get results',datetime.datetime.now())
#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

test[['id','unit_sales']].to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

In [7]:
results = pd.read_csv('model2_test.csv')

In [9]:
results

,Unnamed: 0,id,date,store_nbr,item_nbr,onpromotion,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais,mawk,madw,semax,semin
0,0,125497040,2017-08-16,1,96995,False,2,0.400000,0.454545,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.844024,0.693147,0.693147,0.693147
1,1,125497041,2017-08-16,1,99197,False,2,0.477082,0.433203,0.492276,0.500000,0.333333,0.000000,NaN,NaN,0.455143,1.010969,0.977616,1.098612,0.693147
2,2,125497042,2017-08-16,1,103501,False,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,125497043,2017-08-16,1,103520,False,2,0.588597,0.602354,0.636867,0.675055,0.701263,0.861676,1.000000,NaN,0.675055,1.114774,1.175135,1.791759,0.693147
4,4,125497044,2017-08-16,1,103665,False,2,0.491967,0.496382,0.493545,0.533379,0.572899,0.546085,1.000000,1.000000,0.539732,1.320085,1.282500,1.945910,0.693147
5,5,125497045,2017-08-16,1,105574,False,2,0.477753,0.417475,0.431301,0.411298,0.383807,0.454473,0.692712,0.606840,0.442887,1.693974,1.951031,2.484907,1.386294
6,6,125497046,2017-08-16,1,105575,False,2,0.470809,0.491419,0.437952,0.444527,0.381135,0.432231,0.481280,0.808477,0.457668,2.258817,2.432405,2.833213,1.791759
7,7,125497047,2017-08-16,1,105576,False,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,125497048,2017-08-16,1,105577,False,2,0.615874,0.560554,0.595783,0.682209,0.832321,0.883007,0.707519,0.415037,0.649042,1.022971,1.096418,1.609438,0.693147
9,9,125497049,2017-08-16,1,105693,False,2,0.626292,0.850627,0.847943,0.777778,0.750000,0.750000,1.000000,1.000000,0.812861,0.852271,0.939412,1.098612,0.693147


In [16]:
test = pd.read_csv('model2_test.csv')

In [17]:
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test['unit_sales'] = test['semax'] - test['unit_sales'] * (test['semax'] - test['semin'])
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']



test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#---------------------------------------------------------
print('get results',datetime.datetime.now())
#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

test[['id','unit_sales']].to_csv('model2.csv.gz', index=False, float_format='%.3f', compression='gzip')

get results 2017-11-26 23:50:54.487263
